In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('./csv/db_complementary_filter.csv')

X = df.drop(columns=['activity'])
y = df['activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

import tensorflow as tf
from keras import layers, models
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()  
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.fit_transform(y_test)


model = models.Sequential([
    layers.Input(shape=(24,)),  
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3), 
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                │ (None, 128)            │         3,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,106 (47.29 KB)

 Trainable params: 12,106 (47.29 KB)

 Non-trainable params: 0 (0.00 B)

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

No GPU found, TensorFlow is using the CPU.


In [43]:
model = model.fit(X_train, y_train_encoded, epochs=50, validation_data=(X_test, y_test_encoded), batch_size=32)

import matplotlib.pyplot as plt

# Plotting training & validation accuracy values
plt.plot(model.model['accuracy'])
plt.plot(model.model['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plotting training & validation loss values
plt.plot(model.model['loss'])
plt.plot(model.model['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Epoch 1/50
8968/8968 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.6136 - loss: 0.9898 - val_accuracy: 0.7478 - val_loss: 0.5944
Epoch 2/50
8968/8968 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.7333 - loss: 0.6403 - val_accuracy: 0.7651 - val_loss: 0.5362
Epoch 3/50
8968/8968 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.7519 - loss: 0.5893 - val_accuracy: 0.7834 - val_loss: 0.5042
Epoch 4/50
8968/8968 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.7595 - loss: 0.5675 - val_accuracy: 0.7846 - val_loss: 0.4934
Epoch 5/50
8968/8968 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.7658 - loss: 0.5473 - val_accuracy: 0.7903 - val_loss: 0.4821
Epoch 6/50
8968/8968 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.7704 - loss: 0.5404 - val_accuracy: 0.7931 - val_loss: 0.4761
Epoch 7/50
8968/8968 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step - accuracy: 0.7706 - loss: 0.5345 - val_accuracy: 0.7946 - val_loss: 0.4723
Epoch 8/50
8968/8968 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.7714 - loss: 0

TypeError: 'Sequential' object is not subscriptable

TypeError: 'Sequential' object is not subscriptable